In [1]:
from rodan_pretrained import RodanPretrained
model = RodanPretrained(pretrained_lr=1e-4, my_layers_lr=1e-3, warmup_steps = 10000)

In [2]:
from dataloading2 import nanopore_datamodule
from data_utils.split_methods import get_kfold_split_func, get_default_split
dm = nanopore_datamodule(
    pos_files='pos_2022', 
    neg_files='neg_2022', 
    verbose=1, 
    batch_size=64, 
    valid_limit=10000, 
    workers=32,
    split_method=get_kfold_split_func(total_k=5, current_k=2),
    window=4096,
)

valid files indicies
[0, 2, 22, 28, 29, 33, 35, 36, 38, 63, 64, 98, 109, 121, 122, 126, 127, 130, 133, 135, 143, 148, 149, 152, 157, 164, 166, 167, 168, 170, 172, 181, 210, 217, 218, 220, 221, 225, 226, 230, 233, 237, 241, 242, 244, 245, 246, 247, 255, 256, 259, 260, 261, 262, 273]
[0, 6, 11, 14, 23, 43, 50, 60, 62, 63, 72, 73, 85, 89, 98, 100, 106, 110, 114, 119, 130, 131, 142, 148, 155, 158, 162, 167, 168, 169, 172, 176, 188, 192, 196, 197, 201, 203, 204, 207, 209, 215, 222, 223, 230, 239, 243, 245, 255, 261, 265, 266, 268, 273, 274, 280, 297, 305, 306, 307, 308, 312, 314, 319, 321, 327, 328, 332, 333, 346, 347, 352, 355, 357, 364, 370, 379, 386, 390, 392, 393, 396, 398, 403, 404, 405, 413, 417, 419, 420, 422, 423, 432, 437, 443, 444, 456, 458, 459, 466, 485, 486, 501, 505]
train files indicies
[1, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24, 25, 26, 27, 30, 31, 32, 34, 37, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58,

In [3]:
import pytorch_lightning as pl
from pytorch_lightning.loggers import CometLogger
from pytorch_lightning.callbacks import ModelCheckpoint

experiment_name = 'rodan_2022_kfold_2of5'
checkpoint_callback = ModelCheckpoint(dirpath=f"checkpoints_pl/{experiment_name}", save_top_k=2, monitor="valid_loss")
logger = CometLogger(api_key="TEVQbgxxvilM1WdTyqZLJ57ac", project_name='RNAModif', experiment_name=experiment_name) 
trainer= pl.Trainer(
    max_steps = 1000000, logger=logger, accelerator='gpu',
    auto_lr_find=False, val_check_interval=500,  
    log_every_n_steps=500, benchmark=True, precision=16,
    callbacks=[checkpoint_callback])#, profiler="simple")

trainer.fit(model, dm)


CometLogger will be initialized in online mode
Using 16bit native Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type            | Params
------------------------------------------
0 | model | Sequential      | 10.9 M
1 | acc   | Accuracy        | 0     
2 | cm    | ConfusionMatrix | 0     
------------------------------------------
10.9 M    Trainable params
0         Non-trainable params
10.9 M    Total params
21.760    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:219: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torchmetrics/utilities/prints.py:36: UserWarning: 2 nan values found in confusion matrix have been replaced with zeros.
  warnings.warn(*args, **kwargs)


Training: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
COMET WARNING: Comet has disabled auto-logging functionality as it has been imported after the following ML modules: torch. Metrics and hyperparameters can still be logged using comet_ml.log_metrics() and comet_ml.log_parameters()
COMET WARNING: As you are running in a Jupyter environment, you will need to call `experiment.end()` when finished to ensure all metrics and code are logged before exiting.
COMET INFO: Experime

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/home/jovyan/my-conda-envs/RNAmodEnvRod/lib/python3.8/site-packages/pytorch_lightning/trainer/trainer.py:657: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")
